In [2]:
# Unclear if you have to run this everytime but if you get a sodapy not available error  
!pip install sodapy

In [0]:
# This will take a few minutes
# Documentation, code snippet, and token information: https://dev.socrata.com/foundry/datacatalog.cookcountyil.gov/apwk-dzx8
# This is based heavily on an NYU presentation from Nicholas Wolf and Vicky Steeves-
# Python for Harvesting Data on the Web - https://nyu-dataservices.github.io/DataHarvesting-Python/
from sodapy import Socrata
import pandas as pd

client = Socrata('datacatalog.cookcountyil.gov',
                  'UEcAAUQpAzK6Dl3x7dYTCc5cc',
                  username="kelseymarkey@gmail.com",
                  password="Idsproject1!")

init = client.get("7mck-ehwz", limit=1000000)
disp = client.get("apwk-dzx8", limit=900000)
sent = client.get("tg8v-tm6u", limit=300000)

initiation = pd.DataFrame.from_records(init)
disposition = pd.DataFrame.from_records(disp)
sentencing = pd.DataFrame.from_records(sent)

In [4]:
# Check RAM
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=ef761ed68d7d05fb43c4cd17fb51e57089fc43d1132ad33c11f657221371cc28
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 22.8 GB  | Proc size: 7.0 GB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


**- Check if values in cols are unique:**

<code> sentencing.aoic.is_unique <code>

**- Check which cells in column NaN:**

<code> len(sentencing.loc[sentencing.age_at_incident.isnull()]) <code>

**- Look at value counts of specific column:**

<code> sentencing.charge_disposition.value_counts() </code>

**- Look at value counts of a column:**
<code> disposition.charge_disposition.value_counts() <code

**- Look into groupby:** https://www.geeksforgeeks.org/python-pandas-dataframe-groupby/ 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Result same for sentencing- there are no unique columns in sentencing or disposition :(
for i in disposition.columns:
    if disposition[i].is_unique:
        print(i)

In [0]:
#Trying to make a unique ID by combining multiple columns
 sentencing['person_charge'] = sentencing['case_participant_id'] + '_'+ sentencing['charge_id'] #simplest version

In [0]:
# Exporting the rows that have a non-unique person_charge to a csv file
# t = sentencing[sentencing.duplicated(subset = 'person_charge', keep= False)]
# t.to_csv('sentencing_nonunique.csv')

In [7]:
sentencing['current_sentence'].value_counts()

True     220677
False      8821
Name: current_sentence, dtype: int64

In [8]:
# Karmen on 11/12 suggested using arrest date, case id, part id, charge count
 sentencing['case_person_charge_arrest'] = sentencing['case_id'] + '_' + sentencing['case_participant_id'] + '_'+ sentencing['charge_id'] + '_'+ sentencing['arrest_date']
 print(sentencing.case_person_charge_arrest.is_unique) # = False 
 len(sentencing.case_person_charge_arrest) - len(set(sentencing.case_person_charge_arrest)) #13165 non unique

False


13165

In [5]:
# Amber OHE 
initiation.head(10)

,case_id,case_participant_id,primary_charge,charge_id,charge_version_id,chapter,act,section,class,aoic,event,event_date,age_at_incident,gender,race,incident_begin_date,arrest_date,law_enforcement_agency,received_date,arraignment_date,updated_offense_category,charge_offense_title,charge_count,incident_city,unit,incident_end_date
0,1224415626448,166726386047,False,428562107745,323270746194,720,5,9-1(a)(2),M,0735100,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,2,NaN,NaN,NaN
1,1224415626448,166726386047,False,428563414424,323271731043,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,3,NaN,NaN,NaN
2,1224415626448,166726386047,False,428564534434,323272622096,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,4,NaN,NaN,NaN
3,1224415626448,166726386047,False,428564907770,323272903482,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,5,NaN,NaN,NaN
4,1224415626448,166726386047,False,428565592221,323273419355,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,6,NaN,NaN,NaN
5,1224415626448,166726386047,False,428565778889,323273560048,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,7,NaN,NaN,NaN
6,1224415626448,166726386047,False,428566027780,323273747638,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,8,NaN,NaN,NaN
7,1224415626448,166726386047,False,428566276671,323273935228,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,9,NaN,NaN,NaN
8,1224415626448,166726386047,False,428566463340,323274075921,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,10,NaN,NaN,NaN
9,1224415626448,166726386047,False,428566650008,323274216614,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,11,NaN,NaN,NaN


In [5]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.21.3)


In [10]:
from numpy import array 
from numpy import argmax 
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder


# checking values in each columns
values = array(initiation['law_enforcement_agency'])
print(values)

['PROMIS Data Conversion' 'PROMIS Data Conversion'
 'PROMIS Data Conversion' ... 'CHICAGO PD' 'CHICAGO PD' 'CHICAGO PD']


In [13]:
# OHE FOR INITIATION
initiation.columns 

Index(['case_id', 'case_participant_id', 'primary_charge', 'charge_id',
       'charge_version_id', 'chapter', 'act', 'section', 'class', 'aoic',
       'event', 'event_date', 'age_at_incident', 'gender', 'race',
       'incident_begin_date', 'arrest_date', 'law_enforcement_agency',
       'received_date', 'arraignment_date', 'updated_offense_category',
       'charge_offense_title', 'charge_count', 'incident_city', 'unit',
       'incident_end_date'],
      dtype='object')

In [0]:
from google.colab import files
initiation.to_csv()
#files.download("initiation.csv")

In [0]:
# specify categorical columns for initiation 

init_cat_cols = ['primary_charge', 'class', 'aoic', 'gender', 'race', 'law_enforcement_agency', 'updated_offense_category', 'charge_offense_title' , 'incident_city',]

In [0]:
'''
Create Function for One Hot Encoding 
Get Dummies 
'''

# get table name and column list to OHE
def ohe_dummies(table, column_list): 
  # new empty list of all dfs with get_dummies 
  new_master_df_list = [table]
  
  # get dummies for every column in list
  for column in column_list: 
    dummy_df = pd.get_dummies(table[column])
    # concatenate column to original table, and save that in a new df
    new_master_df_list.append(dummy_df)
  # concat on columns
  master_df = pd.concat(new_master_df_list, axis=1)
  
  return(master_df)

In [9]:
initiation_master_df = ohe_dummies(initiation, init_cat_cols)
initiation_master_df 

,case_id,case_participant_id,primary_charge,charge_id,charge_version_id,chapter,act,section,class,aoic,event,event_date,age_at_incident,gender,race,incident_begin_date,arrest_date,law_enforcement_agency,received_date,arraignment_date,updated_offense_category,charge_offense_title,charge_count,incident_city,unit,incident_end_date,False,True,1,2,3,4,A,B,C,M,O,P,U,X,...,Schererville,Schiller Park,Sheridan,Skokie,South Barrington,South Bend,South Chicago Heights,South Holland,Spring Grove,Springfield,St Charles,Steger,Steward,Stickney,Stone Park,Streamwood,Summit,Summit Argo,Thornton,Tinley Park,University Park,Vernon Hills,Warren,Waukegan,West Chicago,West Dundee,Westchester,Western Sprgs,Western Springs,Westmont,Wheaton,Wheeling,Willow Spgs,Willow Springs,Willowbrook,Wilmette,Winnetka,Wood Dale,Woodstock,Worth
0,1224415626448,166726386047,False,428562107745,323270746194,720,5,9-1(a)(2),M,0735100,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,2,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1224415626448,166726386047,False,428563414424,323271731043,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,3,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1224415626448,166726386047,False,428564534434,323272622096,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,4,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1224415626448,166726386047,False,428564907770,323272903482,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,5,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1224415626448,166726386047,False,428565592221,323273419355,720,5,9-1(a)(3),M,0735200,Preliminary Hearing,6/20/2011 12:00:00 AM,22,Male,Black,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,FIRST DEGREE MURDER,6,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878464,2460008195173,700626591143,True,540209865909,409062376277,720,5,24-1.6(a)(1),4,0017862,NaN,NaN,30,Male,White [Hispanic or Latino],7/24/2019 12:00:00 AM,7/24/2019 10:53:00 PM,CHICAGO PD,7/25/2019 12:00:00 AM,NaN,UUW - Unlawful Use of Weapon,AGGRAVATED UNLAWFUL USE OF WEAPON,1,Chicago,NaN,NaN,0,1,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
878465,2460009845393,700627267556,True,540210114801,409062563867,720,5,24-1.1(a),3,1214500,NaN,NaN,39,Male,Black,7/24/2019 12:00:00 AM,7/24/2019 10:30:00 PM,CHICAGO PD,7/25/2019 12:00:00 AM,NaN,UUW - Unlawful Use of Weapon,UNLAWFUL USE OR POSSESSION OF A WEAPON BY A FELON,1,Chicago,District 2 - Wentworth,NaN,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [10]:
initiation_master_df.columns

Index([            'case_id', 'case_participant_id',      'primary_charge',
                 'charge_id',   'charge_version_id',             'chapter',
                       'act',             'section',               'class',
                      'aoic',
       ...
                   'Wheaton',            'Wheeling',         'Willow Spgs',
            'Willow Springs',         'Willowbrook',            'Wilmette',
                  'Winnetka',           'Wood Dale',           'Woodstock',
                     'Worth'],
      dtype='object', length=4396)

In [0]:
# save to csv but file is too big we run out of ram 
#initiation_master_df.to_csv()

In [18]:
# OHE FOR DISPOSITION
disposition.head(20)

,case_id,case_participant_id,primary_charge,charge_id,charge_version_id,offense_title,chapter,act,section,class,aoic,dispo_date,charge_disposition,judge,court_name,court_facility,age_at_incident,gender,race,offense_type,incident_begin_date,arrest_date,law_enforcement_agency,received_date,arraignment_date,updated_offense_category,charge_count,charge_disposition_reason,incident_end_date,incident_city,unit
0,841463847080,139918426161,False,62512796394,58385258541,PROMIS Conversion,38,-,9-1(a)(2),X,0000001607,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,FIRST DEGREE MURDER,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,2,NaN,NaN,NaN,NaN
1,841463847080,139918426161,False,62520574241,62814780021,PROMIS Conversion,38-33A-2\I,NaN,NaN,X,0000002646,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,ARMED VIOL (CAT I WPN),8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,3,NaN,NaN,NaN,NaN
2,841463847080,139918426161,False,62520636464,66251011171,PROMIS Conversion,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,FIRST DEGREE MURDER,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,4,NaN,NaN,NaN,NaN
3,841463847080,139918426161,False,62520698687,66247728342,PROMIS Conversion,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,FIRST DEGREE MURDER,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,5,NaN,NaN,NaN,NaN
4,841463847080,139918426161,False,62520760909,66247869034,PROMIS Conversion,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,FIRST DEGREE MURDER,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,6,NaN,NaN,NaN,NaN
5,841463847080,139918426161,False,62520823132,66247400059,PROMIS Conversion,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,FIRST DEGREE MURDER,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,7,NaN,NaN,NaN,NaN
6,841463847080,139918426161,False,62677437859,48996507637,PROMIS Conversion,38-10-1-A(1),NaN,NaN,2,0000001642,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,KIDNAPPING,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,8,NaN,NaN,NaN,NaN
7,841463847080,139918426161,False,62512858617,47312556935,PROMIS Conversion,38-10-2-A(3),NaN,NaN,1,0000001655,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,AGGRAVATED KIDNAPPING,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,9,NaN,NaN,NaN,NaN
8,841463847080,139918426161,False,62541356648,47207318810,PROMIS Conversion,38-10-2-A(5),NaN,NaN,1,0000001657,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,AGGRAVATED KIDNAPPING,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,10,NaN,NaN,NaN,NaN
9,841463847080,139918426161,False,62677500082,51590833782,PROMIS Conversion,38-10-3-A,NaN,NaN,4,0000001660,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,UNLAWFUL RESTRAINT,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8

In [19]:
disposition.columns

Index(['case_id', 'case_participant_id', 'primary_charge', 'charge_id',
       'charge_version_id', 'offense_title', 'chapter', 'act', 'section',
       'class', 'aoic', 'dispo_date', 'charge_disposition', 'judge',
       'court_name', 'court_facility', 'age_at_incident', 'gender', 'race',
       'offense_type', 'incident_begin_date', 'arrest_date',
       'law_enforcement_agency', 'received_date', 'arraignment_date',
       'updated_offense_category', 'charge_count', 'charge_disposition_reason',
       'incident_end_date', 'incident_city', 'unit'],
      dtype='object')

In [0]:
disposition_ohe_cols = ['primary_charge', 'offense_title', 'offense_title' , 'judge',
       'court_name', 'court_facility', 'gender', 'race',
       'offense_type','law_enforcement_agency', 'charge_disposition_reason']

In [21]:
disposition_master_df = ohe_dummies(disposition, disposition_ohe_cols)
disposition_master_df 

,case_id,case_participant_id,primary_charge,charge_id,charge_version_id,offense_title,chapter,act,section,class,aoic,dispo_date,charge_disposition,judge,court_name,court_facility,age_at_incident,gender,race,offense_type,incident_begin_date,arrest_date,law_enforcement_agency,received_date,arraignment_date,updated_offense_category,charge_count,charge_disposition_reason,incident_end_date,incident_city,unit,False,True,Aggravated Assault Police Officer,Aggravated Assault Police Officer Firearm,Aggravated Battery,Aggravated Battery Police Officer,Aggravated Battery Police Officer Firearm,Aggravated Battery With A Firearm,Aggravated DUI,...,WHEELING PD,WHEELING POLICE DEPARTMENT,WILLOW SPRINGS PD,WILLOW SPRINGS POLICE DEPARTMENT,WILMETTE PD,WILMETTE POLICE DEPARTMENT,WINNETKA PD,WINNETKA POLICE DEPARTMENT,WOODRIDGE PD,WORTH PD,WORTH POLICE DEPARTMENT,AONIC GJ,Adjudicated Minor,Bond Forfeiture,Complaining Witness No Prosecution,Complaining Witness Not in Court,DDPP Graduate,DGS Graduation,Deferred Prosecution Program Completed,Drug Court Graduate,INDICTMENT,Judgement & Conviction Vacated,Mental Health Graduate,Misdemeanor Amount,Motion to Quash Arrest & Suppress Evidence/Sustained,Negative Lab,No Lab,Nolle - AONIC,PG to Misdemeanor,PG to Other Count/s,Proceeding on Other Case/s,Proceeding on Other Count/s,RAP,Re-Indictment,Sent to GJ,TERM,Transfered to Misdemeanor Court,Used in Aggravation,Veteran's Court Graduate,Warrant Quashed/Recalled
0,841463847080,139918426161,False,62512796394,58385258541,PROMIS Conversion,38,-,9-1(a)(2),X,0000001607,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,FIRST DEGREE MURDER,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,2,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,841463847080,139918426161,False,62520574241,62814780021,PROMIS Conversion,38-33A-2\I,NaN,NaN,X,0000002646,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,ARMED VIOL (CAT I WPN),8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,3,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,841463847080,139918426161,False,62520636464,66251011171,PROMIS Conversion,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,FIRST DEGREE MURDER,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,4,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,841463847080,139918426161,False,62520698687,66247728342,PROMIS Conversion,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,FIRST DEGREE MURDER,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,5,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,841463847080,139918426161,False,62520760909,66247869034,PROMIS Conversion,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Nolle On Remand,John Mannion,District 6 - Markham,Markham Courthouse,27,Male,Black,FIRST DEGREE MURDER,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,Homicide,6,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [22]:
sentencing.head(20)

,case_id,case_participant_id,charge_id,charge_version_id,primary_charge,offense_title,chapter,act,section,class,aoic,dispo_date,sentence_phase,sentence_date,sentence_judge,sentence_type,commitment_type,charge_disposition,court_name,court_facility,length_of_case_in_days,age_at_incident,gender,race,offense_type,incident_begin_date,arrest_date,law_enforcement_agency,received_date,arraignment_date,current_sentence,updated_offense_category,charge_count,commitment_term,commitment_unit,incident_end_date,charge_disposition_reason,incident_city,unit
0,841463847080,139918426161,62512796394,58385258541,False,FIRST DEGREE MURDER,38,-,9-1(a)(2),X,0000001607,12/17/2014 12:00:00 AM,Original Sentencing,6/2/1986 12:00:00 AM,John Mannion,Conversion,Natural Life,Nolle On Remand,District 6 - Markham,Markham Courthouse,619,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,True,Homicide,2,NaN,NaN,NaN,NaN,NaN,NaN
1,841463847080,139918426161,62520636464,66251011171,False,FIRST DEGREE MURDER,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Original Sentencing,6/2/1986 12:00:00 AM,John Mannion,Conversion,Natural Life,Nolle On Remand,District 6 - Markham,Markham Courthouse,619,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,True,Homicide,4,NaN,NaN,NaN,NaN,NaN,NaN
2,841463847080,139918426161,62520698687,66247728342,False,FIRST DEGREE MURDER,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Original Sentencing,6/2/1986 12:00:00 AM,John Mannion,Conversion,Natural Life,Nolle On Remand,District 6 - Markham,Markham Courthouse,619,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,True,Homicide,5,NaN,NaN,NaN,NaN,NaN,NaN
3,841463847080,139918426161,62520885355,49226399479,False,HOME INVASION,38-12-11-A(1),NaN,NaN,X,0000001846,12/17/2014 12:00:00 AM,Amended/Corrected Sentencing,10/16/2014 12:00:00 AM,Clayton Jay Crane,Prison,Illinois Department of Corrections,Plea Of Guilty,District 6 - Markham,Markham Courthouse,10982,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,True,Homicide,13,30,Year(s),NaN,NaN,NaN,NaN
4,841463847080,139918426161,62520885355,49226399479,False,HOME INVASION,38-12-11-A(1),NaN,NaN,X,0000001846,12/17/2014 12:00:00 AM,Original Sentencing,6/2/1986 12:00:00 AM,John Mannion,Conversion,Illinois Department of Corrections,Plea Of Guilty,District 6 - Markham,Markham Courthouse,619,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,False,Homicide,13,030,Year(s),NaN,NaN,NaN,NaN
5,841463847080,139918426161,62512920840,49329527214,False,HOME INVASION,38-12-11-A(2),NaN,NaN,X,0000001847,12/17/2014 12:00:00 AM,Original Sentencing,6/2/1986 12:00:00 AM,John Mannion,Conversion,Illinois Department of Corrections,Nolle On Remand,District 6 - Markham,Markham Courthouse,619,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,True,Homicide,14,030,Year(s),NaN,NaN,NaN,NaN
6,841463847080,139918426161,62520947578,56358861853,False,ARMED ROBBERY,38-18-2-A,NaN,NaN,X,0000002150,12/17/2014 12:00:00 AM,Amended/Corrected Sentencing,10/16/2014 12:00:00 AM,Clayton Jay Crane,Prison,Illinois Department of Corrections,Plea Of Guilty,District 6 - Markham,Markham Courthouse,10982,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,True,Homicide,15,30,Year(s),NaN,NaN,NaN,NaN
7,841463847080,139918426161,62520947578,56358861853,False,ARMED ROBBERY,38-18-2-A,NaN,NaN,X,0000002150,12/17/2014 12:00:00 AM,Original Sentencing,6/2/1986 12:00:00 AM,John Mannion,Conversion,Illinois

In [23]:
# OHE for sentencing

sentencing.columns

Index(['case_id', 'case_participant_id', 'charge_id', 'charge_version_id',
       'primary_charge', 'offense_title', 'chapter', 'act', 'section', 'class',
       'aoic', 'dispo_date', 'sentence_phase', 'sentence_date',
       'sentence_judge', 'sentence_type', 'commitment_type',
       'charge_disposition', 'court_name', 'court_facility',
       'length_of_case_in_days', 'age_at_incident', 'gender', 'race',
       'offense_type', 'incident_begin_date', 'arrest_date',
       'law_enforcement_agency', 'received_date', 'arraignment_date',
       'current_sentence', 'updated_offense_category', 'charge_count',
       'commitment_term', 'commitment_unit', 'incident_end_date',
       'charge_disposition_reason', 'incident_city', 'unit'],
      dtype='object')

In [0]:
sentencing_dum_cols = ['primary_charge', 'offense_title', 'sentence_phase', 'sentence_judge', 'sentence_type', 'commitment_type',
       'charge_disposition', 'gender', 'race',
       'offense_type',  ]

In [25]:
sentencing_master_df = ohe_dummies(sentencing, sentencing_dum_cols)
sentencing_master_df 

,case_id,case_participant_id,charge_id,charge_version_id,primary_charge,offense_title,chapter,act,section,class,aoic,dispo_date,sentence_phase,sentence_date,sentence_judge,sentence_type,commitment_type,charge_disposition,court_name,court_facility,length_of_case_in_days,age_at_incident,gender,race,offense_type,incident_begin_date,arrest_date,law_enforcement_agency,received_date,arraignment_date,current_sentence,updated_offense_category,charge_count,commitment_term,commitment_unit,incident_end_date,charge_disposition_reason,incident_city,unit,False,...,Hate Crimes,Home Invasion,Homicide,Human Trafficking,Identity Theft,Impersonating Police Officer,Intimidation,Kidnapping,Major Accidents,Narcotics,Obstructing Justice,Official Misconduct,Other Offense,PROMIS Conversion,Pandering,Perjury,Police Shooting,Possession Of Burglary Tools,Possession of Contraband in Penal Institution,Possession of Explosives,Possession of Shank in Penal Institution,Possession of Stolen Motor Vehicle,Prostitution,Reckless Discharge of Firearm,Reckless Homicide,Residential Burglary,Retail Theft,Robbery,Sex Crimes,Stalking,Tampering,Theft,Theft by Deception,UUW - Unlawful Use of Weapon,Unlawful Restraint,Vehicular Hijacking,Vehicular Invasion,Violate Bail Bond,Violation Order Of Protection,Violation of Sex Offender Registration
0,841463847080,139918426161,62512796394,58385258541,False,FIRST DEGREE MURDER,38,-,9-1(a)(2),X,0000001607,12/17/2014 12:00:00 AM,Original Sentencing,6/2/1986 12:00:00 AM,John Mannion,Conversion,Natural Life,Nolle On Remand,District 6 - Markham,Markham Courthouse,619,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,True,Homicide,2,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,841463847080,139918426161,62520636464,66251011171,False,FIRST DEGREE MURDER,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Original Sentencing,6/2/1986 12:00:00 AM,John Mannion,Conversion,Natural Life,Nolle On Remand,District 6 - Markham,Markham Courthouse,619,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,True,Homicide,4,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,841463847080,139918426161,62520698687,66247728342,False,FIRST DEGREE MURDER,38,-,9-1(a)(3),X,0000001608,12/17/2014 12:00:00 AM,Original Sentencing,6/2/1986 12:00:00 AM,John Mannion,Conversion,Natural Life,Nolle On Remand,District 6 - Markham,Markham Courthouse,619,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,True,Homicide,5,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,841463847080,139918426161,62520885355,49226399479,False,HOME INVASION,38-12-11-A(1),NaN,NaN,X,0000001846,12/17/2014 12:00:00 AM,Amended/Corrected Sentencing,10/16/2014 12:00:00 AM,Clayton Jay Crane,Prison,Illinois Department of Corrections,Plea Of Guilty,District 6 - Markham,Markham Courthouse,10982,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,True,Homicide,13,30,Year(s),NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,841463847080,139918426161,62520885355,49226399479,False,HOME INVASION,38-12-11-A(1),NaN,NaN,X,0000001846,12/17/2014 12:00:00 AM,Original Sentencing,6/2/1986 12:00:00 AM,John Mannion,Conversion,Illinois Department of Corrections,Plea Of Guilty,District 6 - Markham,Markham Courthouse,619,27,Male,Black,PROMIS Conversion,8/9/1984 12:00:00 AM,8/15/1984 12:00:00 AM,CHICAGO POLICE DEPT,8/15/1984 12:00:00 AM,9/21/1984 12:00:00 AM,False,Homicide,13,030,Year(s),NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,

In [0]:
sentencing_master_df.to_csv()